In [2]:
pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━

In [4]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm

# PyTorch và các thư viện liên quan
import torch
import timm
from torchvision import transforms

# Scikit-learn và AutoGluon
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, log_loss, roc_auc_score
from autogluon.tabular import TabularPredictor

# --- BƯỚC 0: CÀI ĐẶT BAN ĐẦU ---

# Xác định thiết bị (sử dụng GPU nếu có)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")


# --- BƯỚC 1: TẢI DỮ LIỆU VÀ ĐỊNH NGHĨA ĐƯỜNG DẪN ---
print("Đang tải dữ liệu train.csv...")
# Giả sử file train.csv và thư mục train_images nằm cùng cấp với notebook
train_df = pd.read_csv('/kaggle/input/paddy-disease-classification/train.csv')
image_base_path = '/kaggle/input/paddy-disease-classification/train_images'

# Tạo đường dẫn đầy đủ cho mỗi ảnh
train_df['image_path'] = train_df.apply(lambda row: os.path.join(image_base_path, row['label'], row['image_id']), axis=1)
print(f"Đã tải {len(train_df)} mẫu.")


# --- BƯỚC 2: THIẾT LẬP MÔ HÌNH TRÍCH XUẤT ĐẶC TRƯNG ---
print("Đang thiết lập mô hình EfficientNet-B0 để trích xuất đặc trưng...")
# Tải mô hình EfficientNet-B0 đã huấn luyện trước từ thư viện timm
# num_classes=0 sẽ loại bỏ lớp phân loại cuối cùng, biến nó thành một feature extractor
feature_extractor = timm.create_model('efficientnet_b0', pretrained=True, num_classes=0).to(device)
feature_extractor.eval() # Chuyển mô hình sang chế độ đánh giá

# Lấy các thiết lập chuẩn hóa ảnh mặc định của mô hình
config = timm.data.resolve_data_config({}, model=feature_extractor)
transform = timm.data.create_transform(**config)


# --- BƯỚC 3: CHẠY QUÁ TRÌNH TRÍCH XUẤT ĐẶC TRƯNG ---
# Đây là bước tốn nhiều thời gian nhất, đặc biệt nếu không có GPU
print(f"Bắt đầu trích xuất đặc trưng cho {len(train_df)} ảnh. Quá trình này có thể mất vài phút...")

features_list = []
with torch.no_grad(): # Tắt tính toán gradient để tăng tốc
    for image_path in tqdm(train_df['image_path']):
        img = Image.open(image_path).convert('RGB')
        img_tensor = transform(img).unsqueeze(0).to(device) # Chuyển ảnh thành tensor và đưa lên device
        
        # Trích xuất đặc trưng
        features = feature_extractor(img_tensor)
        
        # Chuyển về numpy array trên CPU và thêm vào danh sách
        features_list.append(features.cpu().numpy().flatten())

# Chuyển danh sách các đặc trưng thành một DataFrame
feature_column_names = [f'feature_{i}' for i in range(len(features_list[0]))]
features_df = pd.DataFrame(features_list, columns=feature_column_names)
print(f"Trích xuất đặc trưng hoàn tất. Mỗi ảnh có {len(feature_column_names)} đặc trưng.")


# --- BƯỚC 4: KẾT HỢP DỮ LIỆU TẠO BẢNG CUỐI CÙNG ---
print("Đang kết hợp đặc trưng ảnh và metadata...")
# Nối dataframe gốc với dataframe đặc trưng
final_df = pd.concat([train_df.reset_index(drop=True), features_df], axis=1)

# Bỏ các cột không cần thiết cho việc huấn luyện tabular
final_df = final_df.drop(columns=['image_id', 'image_path'])


# --- BƯỚC 5: CHIA DỮ LIỆU VÀ HUẤN LUYỆN VỚI AUTOGLUON TABULAR ---
label_column = 'label'
print(f"\nChuẩn bị huấn luyện với AutoGluon Tabular. Nhãn dự đoán: '{label_column}'")

# Chia dữ liệu: 80% train, 20% test
train_data, test_data = train_test_split(final_df, test_size=0.2, stratify=final_df[label_column], random_state=42)
print(f"Kích thước tập huấn luyện: {train_data.shape}")
print(f"Kích thước tập đánh giá: {test_data.shape}")

# Khởi tạo TabularPredictor
save_path = 'ag_paddy_tabular_models' # Thư mục lưu mô hình
predictor = TabularPredictor(label=label_column, path=save_path, eval_metric='accuracy')

# Huấn luyện mô hình
# time_limit: giới hạn thời gian (giây), presets: chất lượng mô hình
print("Bắt đầu huấn luyện...")
predictor.fit(train_data, time_limit=10*60, presets='best_quality')


# --- BƯỚC 6: ĐÁNH GIÁ MÔ HÌNH TRÊN 5 TIÊU CHÍ ---
print("\nĐánh giá mô hình tốt nhất trên tập dữ liệu đánh giá (20%)...")

y_true = test_data[label_column]
y_pred = predictor.predict(test_data)
y_proba = predictor.predict_proba(test_data)

# Tính toán các chỉ số
accuracy = accuracy_score(y_true, y_pred)
logloss = log_loss(y_true, y_proba)
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')
roc_auc = roc_auc_score(y_true, y_proba, multi_class='ovr')

# In kết quả
print("-" * 50)
print("--- KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH ---")
print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ Log Loss: {logloss:.4f}")
print(f"✅ F1 Macro: {f1_macro:.4f}")
print(f"✅ F1 Weighted: {f1_weighted:.4f}")
print(f"✅ ROC AUC (One-vs-Rest): {roc_auc:.4f}")
print("-" * 50)

# Hiển thị bảng xếp hạng các mô hình mà AutoGluon đã thử
print("\nBảng xếp hạng các mô hình:")
leaderboard = predictor.leaderboard(test_data, silent=True)
print(leaderboard)

Sử dụng thiết bị: cuda
Đang tải dữ liệu train.csv...
Đã tải 10407 mẫu.
Đang thiết lập mô hình EfficientNet-B0 để trích xuất đặc trưng...


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Bắt đầu trích xuất đặc trưng cho 10407 ảnh. Quá trình này có thể mất vài phút...


  0%|          | 0/10407 [00:00<?, ?it/s]

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       28.87 GB / 31.35 GB (92.1%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacke

Trích xuất đặc trưng hoàn tất. Mỗi ảnh có 1280 đặc trưng.
Đang kết hợp đặc trưng ảnh và metadata...

Chuẩn bị huấn luyện với AutoGluon Tabular. Nhãn dự đoán: 'label'
Kích thước tập huấn luyện: (8325, 1283)
Kích thước tập đánh giá: (2082, 1283)
Bắt đầu huấn luyện...


	Running DyStack sub-fit in a ray process to avoid memory leakage. Enabling ray logging (enable_ray_logging=True). Specify `ds_args={'enable_ray_logging': False}` if you experience logging issues.
2025-07-03 05:36:08,538	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
		Context path: "/kaggle/working/ag_paddy_tabular_models/ds_sub_fit/sub_fit_ho"
(_dystack pid=329) Running DyStack sub-fit ...
(_dystack pid=329) Beginning AutoGluon training ... Time limit = 146s
(_dystack pid=329) AutoGluon will save models to "/kaggle/working/ag_paddy_tabular_models/ds_sub_fit/sub_fit_ho"
(_dystack pid=329) Train Data Rows:    7400
(_dystack pid=329) Train Data Columns: 1282
(_dystack pid=329) Label Column:       label
(_dystack pid=329) Problem Type:       multiclass
(_dystack pid=329) Preprocessing data ...
(_dystack pid=329) Train Data Class Count: 10
(_dystack pid=329) Using Feature Generators to preprocess the data ...
(_dystack pid=329) Fitting Au


Đánh giá mô hình tốt nhất trên tập dữ liệu đánh giá (20%)...
--------------------------------------------------
--- KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH ---
✅ Accuracy: 0.9625
✅ Log Loss: 0.6020
✅ F1 Macro: 0.9585
✅ F1 Weighted: 0.9625
✅ ROC AUC (One-vs-Rest): 0.9976
--------------------------------------------------

Bảng xếp hạng các mô hình:
                    model  score_test  score_val eval_metric  pred_time_test  \
0  NeuralNetFastAI_BAG_L2    0.963977   0.960360    accuracy        3.377850   
1     WeightedEnsemble_L3    0.962536   0.962402    accuracy        3.534894   
2     WeightedEnsemble_L2    0.958694   0.959760    accuracy        1.411825   
3  NeuralNetFastAI_BAG_L1    0.947166   0.936336    accuracy        0.764049   
4   KNeighborsDist_BAG_L1    0.944765   0.947267    accuracy        0.462926   
5       LightGBMXT_BAG_L2    0.941402   0.947267    accuracy        2.447479   
6   KNeighborsUnif_BAG_L1    0.932757   0.940060    accuracy        0.463795   
7       LightGBMXT_BAG_L